#Aplicación de PCA

En este notebook veremos la aplicación de PCA a un set de datos

###Cargamos las librerias necesarias

In [ ]:
# Insertar cuantos bloques de código y markdown consideren necesarios

import numpy as np # Programación vectorial

import pandas as pd # Estructura y manipulación de datos
import seaborn as sns
from seaborn import boxplot
from numpy import array

# Gráficos

import matplotlib.pyplot as plt # Visualización
from matplotlib import style


# Preprocesado y modelado
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

# Configuración matplotlib
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')


In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

###Cargamos el data set.

El set de datos USArrests contiene el porcentaje de asaltos (Assault), asesinatos (Murder) y secuestros (Rape) por cada 100,000 habitantes para cada uno de los 50 estados de USA (1973). Además, también incluye el porcentaje de la población de cada estado que vive en zonas rurales (UrbanPoP).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#rt ="/content/drive/MyDrive/Base_clientes_Monopoly THIS.xlsx"
rt="/content/drive/MyDrive/solo_base__clientes_Monopoly.xlsx"

df = pd.read_excel(rt)
df.head(10)


Mounted at /content/drive


###Vemos los tipos de datos.

Todo se ve en orden ya que son numéricos

In [ ]:
df.head()

,Id,Subsegmento,Sexo,Region,Edad,Renta,Antiguedad,Internauta,Adicional,Dualidad,...,ColMx_T01,PagoNac_T01,PagoInt_T01,EeccNac_T01,EeccInt_T01,UsoL1_T01,UsoL2_T01,UsoLI_T01,IndRev_T01,target
0,1,160,M,13.0,43,NaN,130,1,1,0,...,0.0,33000,0.0,1099866.0,0.0,1099866.0,15080,0.0,R,0
1,2,160,H,13.0,46,143640.0,69,1,0,0,...,0.0,300000,0.0,214592.0,0.0,214592.0,83596,0.0,R,0
2,3,170,H,13.0,45,929106.0,24,1,1,0,...,0.0,216676,0.0,0.0,0.0,7400.0,0,0.0,T,0
3,4,151,H,13.0,46,172447.0,134,0,1,0,...,0.0,60000,0.0,272762.0,0.0,272762.0,10591,0.0,R,0
4,5,170,H,13.0,46,805250.0,116,0,1,1,...,0.0,272925,0.0,249562.0,0.0,75339.0,377782,0.0,R,0


In [ ]:
df.info()

In [ ]:
df.describe()

Vemos que la escala de los datos es muy diferente. Es necesario escalar los datos. Se deben dejar con media cero y desviación estándar 1.

Por defecto, PCA() centra los valores pero no los escala. Esto es importante ya que, si las variables tienen distinta dispersión, como en este caso, es necesario escalarlas.

## One-hot encoding para transformar las 13 variables categoricas que tiene el dataset

In [ ]:
#  variables categóricas
col_cat=['Sexo','IndRev_T12', 'IndRev_T11', 'IndRev_T10', 'IndRev_T09', 'IndRev_T08', 'IndRev_T07',
                        'IndRev_T06', 'IndRev_T05', 'IndRev_T04', 'IndRev_T03', 'IndRev_T02', 'IndRev_T01']


In [ ]:
# Supongamos que df es tu DataFrame original con las variables categóricas
# y col_cat es la lista de nombres de las variables categóricas

# Seleccionar solo las columnas que son variables categóricas
#df_cat = df[col_cat]

# Aplicar one-hot encoding
df_onehot = pd.get_dummies(col_cat)

# Concatenar el DataFrame one-hot encoding con el DataFrame original
df1 = pd.concat([df, df_onehot], axis=1)

# Eliminar las columnas originales de las variables categóricas
df1 = df.drop(col_cat, axis=1)

# Mostrar el DataFrame final
print(df1.head())


   Id  Subsegmento  Region  Edad     Renta  Antiguedad  Internauta  Adicional  \
0   1          160    13.0    43       0.0         130           1          1   
1   2          160    13.0    46  143640.0          69           1          0   
2   3          170    13.0    45  929106.0          24           1          1   
3   4          151    13.0    46  172447.0         134           0          1   
4   5          170    13.0    46  805250.0         116           0          1   

   Dualidad  Monoproducto  ...  ColL2CC_T01  ColMx_T01  PagoNac_T01  \
0         0             0  ...        29396        0.0        33000   
1         0             0  ...            0        0.0       300000   
2         0             0  ...            0        0.0       216676   
3         0             1  ...            0        0.0        60000   
4         1             0  ...        32028        0.0       272925   

   PagoInt_T01  EeccNac_T01  EeccInt_T01  UsoL1_T01  UsoL2_T01  UsoLI_T01  \
0        

Eliminar columnas innecesarias para el analisis

In [ ]:
df1= df.drop(['Id','CambioPin'], axis=1)

Imputacion datos faltantes

In [ ]:
df1['Renta'] = df1['Renta'].fillna(0)

In [ ]:
# Calcular la media de la variable "Región" (ignorando los valores N
df1['Region'].mode()[0]


13.0

In [ ]:
df1['Region'] = df1['Region'].fillna(13)

In [ ]:

print(df1.isnull().sum().sort_values(ascending=False))

UsoL2_T12        8251
FacCI_T12        8251
FacAN_T12        8251
TxsAN_T12        8251
FacAI_T12        8251
                 ... 
FlgActCOL_T01       0
Fac_T01             0
Txs_T01             0
FacCN_T01           0
target              0
Length: 572, dtype: int64


In [ ]:
tota = df1.isnull().sum().sum()
print(f'Total de valores nulos en el DataFrame: {tota}')

Total de valores nulos en el DataFrame: 1242741


In [ ]:
df2 = df1.dropna()

In [ ]:
df2.isna().sum()

Subsegmento    0
Sexo           0
Region         0
Edad           0
Renta          0
              ..
UsoL1_T01      0
UsoL2_T01      0
UsoLI_T01      0
IndRev_T01     0
target         0
Length: 572, dtype: int64

In [ ]:
df2.isnull().sum().sum()

0

In [ ]:
df2.head()

,Subsegmento,Sexo,Region,Edad,Renta,Antiguedad,Internauta,Adicional,Dualidad,Monoproducto,...,ColMx_T01,PagoNac_T01,PagoInt_T01,EeccNac_T01,EeccInt_T01,UsoL1_T01,UsoL2_T01,UsoLI_T01,IndRev_T01,target
0,160,M,13.0,43,0.0,130,1,1,0,0,...,0.0,33000,0.0,1099866.0,0.0,1099866.0,15080,0.0,R,0
1,160,H,13.0,46,143640.0,69,1,0,0,0,...,0.0,300000,0.0,214592.0,0.0,214592.0,83596,0.0,R,0
2,170,H,13.0,45,929106.0,24,1,1,0,0,...,0.0,216676,0.0,0.0,0.0,7400.0,0,0.0,T,0
3,151,H,13.0,46,172447.0,134,0,1,0,1,...,0.0,60000,0.0,272762.0,0.0,272762.0,10591,0.0,R,0
4,170,H,13.0,46,805250.0,116,0,1,1,0,...,0.0,272925,0.0,249562.0,0.0,75339.0,377782,0.0,R,0


In [ ]:
# Supongamos que df es tu DataFrame original con las variables categóricas
# y col_cat es la lista de nombres de las variables categóricas

# Seleccionar solo las columnas que son variables categóricas
#df_cat = df[col_cat]

# Aplicar one-hot encoding
df_onehot = pd.get_dummies(col_cat)

# Concatenar el DataFrame one-hot encoding con el DataFrame original
df2 = pd.concat([df, df_onehot], axis=1)

# Eliminar las columnas originales de las variables categóricas
df2 = df1.drop(col_cat, axis=1)

# Mostrar el DataFrame final
print(df2.head())


   Subsegmento  Region  Edad     Renta  Antiguedad  Internauta  Adicional  \
0          160    13.0    43       0.0         130           1          1   
1          160    13.0    46  143640.0          69           1          0   
2          170    13.0    45  929106.0          24           1          1   
3          151    13.0    46  172447.0         134           0          1   
4          170    13.0    46  805250.0         116           0          1   

   Dualidad  Monoproducto  Ctacte  ...  ColL2CC_T01  ColMx_T01  PagoNac_T01  \
0         0             0       1  ...        29396        0.0        33000   
1         0             0       1  ...            0        0.0       300000   
2         0             0       1  ...            0        0.0       216676   
3         0             1       0  ...            0        0.0        60000   
4         1             0       1  ...        32028        0.0       272925   

   PagoInt_T01  EeccNac_T01  EeccInt_T01  UsoL1_T01  UsoL2_T01

###Se escalan los datos

In [ ]:
# Se crea primero el objeto Standard Scaler
scaler = StandardScaler()

# Luego "entrenamos" con los datos para obtener los parámetros del escalamiento
scaler.fit(df3)

# Finalmente aplicamos el escalamiento
array_scaler = scaler.transform(df3)

In [ ]:
#Corroboramos que dio resultado
pd.DataFrame(array_scaler).describe()

,0,1,2,3,4,5,6,7,8,9,...,549,550,551,552,553,554,555,556,557,558
count,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,...,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04
mean,-3.966256e-16,-1.328227e-16,1.554344e-16,1.232755e-16,3.483874e-17,-1.728537e-16,-1.473947e-16,6.029781e-18,-1.339951e-17,1.306453e-16,...,3.885859e-17,1.071961e-17,-4.019854e-18,-6.029781e-18,1.875932e-17,-9.463407e-18,-3.215883e-17,1.071961e-17,-1.540944e-17,-1.284678e-16
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,...,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00
min,-1.046070e+00,-2.939945e+00,-1.534836e+00,-1.073372e+00,-1.068198e+00,-1.441940e+00,-6.150790e-01,-7.640273e-01,-2.805825e-01,-3.257748e+00,...,-3.949343e-01,-1.085649e-01,-5.185678e-01,-4.378413e-02,-6.839189e+00,-7.379601e+01,-1.307863e+01,-3.141723e-01,-7.035966e+01,-3.234127e-01
25%,-7.331581e-01,-5.586593e-01,-7.995095e-01,-1.073372e+00,-7.087937e-01,-1.441940e+00,-6.150790e-01,-7.640273e-01,-2.805825e-01,3.069605e-01,...,-3.949343e-01,-1.085649e-01,-4.860103e-01,-4.378413e-02,-5.668166e-01,-7.302657e-02,-5.800451e-01,-3.141633e-01,-7.949237e-02,-3.234127e-01
50%,-3.854784e-01,6.319834e-01,-2.847809e-01,-6.370780e-02,-4.046822e-01,6.935103e-01,-6.150790e-01,-7.640273e-01,-2.805825e-01,3.069605e-01,...,-3.949343e-01,-1.085649e-01,-2.843852e-01,-4.378413e-02,-3.593191e-01,-7.302657e-02,-3.638511e-01,-2.520201e-01,-7.949237e-02,-3.234127e-01
75%,1.005240e+00,6.319834e-01,6.711437e-01,4.969094e-01,4.523594e-01,6.935103e-01,1.625807e+00,1.308854e+00,-2.805825e-01,3.069605e-01,...,-3.245447e-02,-1.085649e-01,1.201950e-01,-4.378413e-02,1.436719e-01,-7.302657e-02,1.500601e-01,-8.468123e-02,-7.949237e-02,-3.234127e-01
max,2.231800e+01,6.319834e-01,4.715440e+00,2.714114e+01,7.723389e+00,6.935103e-01,1.625807e+00,1.308854e+00,3.564014e+00,3.069605e-01,...,2.602861e+01,4.118615e+01,5.611698e+01,1.075882e+02,1.424652e+01,3.666162e+01,1.425628e+01,2.638703e+01,3.539687e+01,3.092025e+00


In [ ]:
# Eliminar columnas con al menos un valor nulo
df3 = df2.dropna(axis=0)

# Mostrar el DataFrame después de eliminar las columnas nulas
print(df3.head())

   Subsegmento  Region  Edad     Renta  Antiguedad  Internauta  Adicional  \
0          160    13.0    43       0.0         130           1          1   
1          160    13.0    46  143640.0          69           1          0   
2          170    13.0    45  929106.0          24           1          1   
3          151    13.0    46  172447.0         134           0          1   
4          170    13.0    46  805250.0         116           0          1   

   Dualidad  Monoproducto  Ctacte  ...  ColL2CC_T01  ColMx_T01  PagoNac_T01  \
0         0             0       1  ...        29396        0.0        33000   
1         0             0       1  ...            0        0.0       300000   
2         0             0       1  ...            0        0.0       216676   
3         0             1       0  ...            0        0.0        60000   
4         1             0       1  ...        32028        0.0       272925   

   PagoInt_T01  EeccNac_T01  EeccInt_T01  UsoL1_T01  UsoL2_T01

In [ ]:
df3.shape

(42422, 559)

In [ ]:
df3.isna().sum().sum()

0

In [ ]:
df4=df3

In [ ]:
# Se crea primero el objeto Standard Scaler
scaler = StandardScaler()

# Luego "entrenamos" con los datos para obtener los parámetros del escalamiento
scaler.fit(datos)

# Finalmente aplicamos el escalamiento
array_scaler = scaler.transform(datos)

###Aplicamos PCA

[Acá](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) puede encontrar información de la función PCA

In [ ]:
# Al no incluir el número de componentes, lo que hace PCA es conservarlos todos
pca = PCA()

In [ ]:
pca.fit(array_scaler)

PCA()

In [ ]:
scaler=StandardScaler()#instantiate
scaler.fit(df3) # calcula la media y estandar para cada dimension
X_scaled=scaler.transform(df3)# transforma los datos a su nueva escala

In [ ]:
pca.fit(X_scaled) # buscar los componentes principales
X_pca=pca.transform(X_scaled)
#revisemos la forma del array
print("shape of X_pca", X_pca.shape)

shape of X_pca (42422, 559)


Veamos cuán buenos predictores son las nuevas dimensiones

In [ ]:
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:160]))
#con 160 componentes tenemos algo mas del 87,7% de varianza explicada

[6.54291627e-02 5.04237219e-02 4.78500687e-02 4.01359450e-02
 3.71706945e-02 2.96268204e-02 1.80382771e-02 1.69507058e-02
 1.43261298e-02 1.38326281e-02 1.29035973e-02 1.14634604e-02
 1.08302014e-02 1.02660693e-02 9.61697454e-03 9.50749319e-03
 8.98855888e-03 8.57854653e-03 8.30466553e-03 7.88047317e-03
 7.70963450e-03 7.56827547e-03 7.26054068e-03 7.15956008e-03
 6.94090248e-03 6.88582889e-03 6.77820141e-03 6.71173612e-03
 6.62282201e-03 6.61080839e-03 6.55846874e-03 6.42846940e-03
 6.41585044e-03 6.27349622e-03 6.16172391e-03 6.06249576e-03
 5.80309336e-03 5.77816318e-03 5.63335517e-03 5.55659799e-03
 5.47812091e-03 5.44747697e-03 5.42756008e-03 5.34151220e-03
 5.25144827e-03 5.22593569e-03 5.12475997e-03 5.04226973e-03
 4.87817004e-03 4.82069862e-03 4.74091714e-03 4.70392350e-03
 4.56094558e-03 4.48776169e-03 4.41703420e-03 4.33700166e-03
 4.27025203e-03 4.17805311e-03 4.11473636e-03 4.06261303e-03
 3.99156054e-03 3.88110557e-03 3.84839499e-03 3.78805759e-03
 3.67256997e-03 3.627134

###Un poco de interpretación:

Una vez entrenado el objeto PCA, pude accederse a toda la información de las componentes creadas.

components_ contiene el valor que definen cada componente (eigenvector). Las filas se corresponden con las componentes principales (ordenadas de mayor a menor varianza explicada). Las filas se corresponden con las variables de entrada.

In [ ]:
# Se combierte el array a dataframe para añadir nombres a los ejes.
pd.DataFrame(
    data    = pca.components_,
    columns = datos.columns,
    index   = ['PC1', 'PC2', 'PC3', 'PC4']
)

TypeError: ignored

Analizar con detalle el vector que forma cada componente puede ayudar a interpretar qué tipo de información recoge cada una de ellas. Por ejemplo, la primera componente es el resultado de la siguiente combinación lineal de las variables originales:

**PC1=0.535899 Murder+0.583184 Assault+0.278191 UrbanPop+0.543432 Rape**


Los pesos asignados en la primera componente a las variables Assault, Murder y Rape son aproximadamente iguales entre ellos y superiores al asignado a UrbanPoP. Esto significa que la primera componente recoge mayoritariamente la información correspondiente a los delitos. En la segunda componente, es la variable UrbanPoP es la que tiene el peso mayor (aunque sea negativo), por lo que se corresponde principalmente con el nivel de urbanización del estado. Si bien en este ejemplo la interpretación de las componentes es bastante clara, no en todos los casos ocurre lo mismo, sobre todo a medida que aumenta el número de variables.

Una vez calculadas las componentes principales, se puede conocer la varianza explicada por cada una de ellas, la proporción respecto al total y la proporción de varianza acumulada. Esta información está almacenada en los atributos **explained_variance_** y **explained_variance_ratio_** del modelo.

In [ ]:
print(pca.explained_variance_ratio_)

[6.54291627e-02 5.04237219e-02 4.78500687e-02 4.01359450e-02
 3.71706945e-02 2.96268204e-02 1.80382771e-02 1.69507058e-02
 1.43261298e-02 1.38326281e-02 1.29035973e-02 1.14634604e-02
 1.08302014e-02 1.02660693e-02 9.61697454e-03 9.50749319e-03
 8.98855888e-03 8.57854653e-03 8.30466553e-03 7.88047317e-03
 7.70963450e-03 7.56827547e-03 7.26054068e-03 7.15956008e-03
 6.94090248e-03 6.88582889e-03 6.77820141e-03 6.71173612e-03
 6.62282201e-03 6.61080839e-03 6.55846874e-03 6.42846940e-03
 6.41585044e-03 6.27349622e-03 6.16172391e-03 6.06249576e-03
 5.80309336e-03 5.77816318e-03 5.63335517e-03 5.55659799e-03
 5.47812091e-03 5.44747697e-03 5.42756008e-03 5.34151220e-03
 5.25144827e-03 5.22593569e-03 5.12475997e-03 5.04226973e-03
 4.87817004e-03 4.82069862e-03 4.74091714e-03 4.70392350e-03
 4.56094558e-03 4.48776169e-03 4.41703420e-03 4.33700166e-03
 4.27025203e-03 4.17805311e-03 4.11473636e-03 4.06261303e-03
 3.99156054e-03 3.88110557e-03 3.84839499e-03 3.78805759e-03
 3.67256997e-03 3.627134

En orden, PC1 explica la varianza total en un 62%

Si sumamos todo, debe dar un 100% (o casi un 100%)

In [ ]:
print(sum(pca.explained_variance_ratio_)*100)

100.00000000000011


Podemos obtener la varianza acumulada simplemente sumando y acumulando lo aportado por cada componente.

In [ ]:
pca.explained_variance_ratio_.cumsum()

array([0.06542916, 0.11585288, 0.16370295, 0.2038389 , 0.24100959,
       0.27063641, 0.28867469, 0.3056254 , 0.31995153, 0.33378415,
       0.34668775, 0.35815121, 0.36898141, 0.37924748, 0.38886446,
       0.39837195, 0.40736051, 0.41593906, 0.42424372, 0.43212419,
       0.43983383, 0.4474021 , 0.45466264, 0.4618222 , 0.46876311,
       0.47564894, 0.48242714, 0.48913887, 0.4957617 , 0.5023725 ,
       0.50893097, 0.51535944, 0.52177529, 0.52804879, 0.53421051,
       0.54027301, 0.5460761 , 0.55185427, 0.55748762, 0.56304422,
       0.56852234, 0.57396982, 0.57939738, 0.58473889, 0.58999034,
       0.59521627, 0.60034103, 0.6053833 , 0.61026147, 0.61508217,
       0.61982309, 0.62452701, 0.62908796, 0.63357572, 0.63799275,
       0.64232975, 0.64660001, 0.65077806, 0.6548928 , 0.65895541,
       0.66294697, 0.66682808, 0.67067647, 0.67446453, 0.6781371 ,
       0.68176423, 0.68530986, 0.68875087, 0.69213663, 0.69550569,
       0.69884978, 0.70211675, 0.70536121, 0.70854901, 0.71169

Esto nos dice por ejemplo, que solo necesitamos  dimensiones (en el nuevo espacio de coordenadas) para explicar el 86.8% de la varianza total.

Una vez hecho esto, podemos transformar los datos originales al espacio nuevo creado.

La transformación es el resultado de multiplicar los vectores que definen cada componente con el valor de las variables.

In [ ]:
# Aplicamos la transformación
proyecciones = pca.transform(df3)
# Lo transformamos a DataFrame, ya que queda como arreglo.
proyecciones = pd.DataFrame(proyecciones,columns = ['PC1', 'PC2', 'PC3', 'PC4'], index = df3.index)
proyecciones.head()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


ValueError: ignored

###Comparemos con el original

In [ ]:
datos.head()

Acá podemos decidir con cuantos componentes nos quedamos y así reducimos la dimensión. Eso dependiendo de con cuanta varianza explicada queremos quedarnos.

Finalmente, se puede reconstruir al espacio original una vez estando en el espacio transformado. Esto se hace con el método **inverse_transform()**. Es importante tener en cuenta que, la reconstrucción, solo será completa si se han incluido todas las componentes.

In [ ]:
recostruccion = pca.inverse_transform(proyecciones)
recostruccion = pd.DataFrame(recostruccion, columns = datos.columns, index = datos.index)
recostruccion.head()

In [ ]:
# Se crea primero el objeto Standard Scaler
scaler = StandardScaler()

# Luego "entrenamos" con los datos para obtener los parámetros del escalamiento
scaler.fit(df4)

# Finalmente aplicamos el escalamiento
array_scaler = scaler.transform(df4)

In [ ]:
#Corroboramos que dio resultado
pd.DataFrame(array_scaler).describe()

,0,1,2,3,4,5,6,7,8,9,...,549,550,551,552,553,554,555,556,557,558
count,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,...,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04,4.242200e+04
mean,-3.966256e-16,-1.328227e-16,1.554344e-16,1.232755e-16,3.483874e-17,-1.728537e-16,-1.473947e-16,6.029781e-18,-1.339951e-17,1.306453e-16,...,3.885859e-17,1.071961e-17,-4.019854e-18,-6.029781e-18,1.875932e-17,-9.463407e-18,-3.215883e-17,1.071961e-17,-1.540944e-17,-1.284678e-16
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,...,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00
min,-1.046070e+00,-2.939945e+00,-1.534836e+00,-1.073372e+00,-1.068198e+00,-1.441940e+00,-6.150790e-01,-7.640273e-01,-2.805825e-01,-3.257748e+00,...,-3.949343e-01,-1.085649e-01,-5.185678e-01,-4.378413e-02,-6.839189e+00,-7.379601e+01,-1.307863e+01,-3.141723e-01,-7.035966e+01,-3.234127e-01
25%,-7.331581e-01,-5.586593e-01,-7.995095e-01,-1.073372e+00,-7.087937e-01,-1.441940e+00,-6.150790e-01,-7.640273e-01,-2.805825e-01,3.069605e-01,...,-3.949343e-01,-1.085649e-01,-4.860103e-01,-4.378413e-02,-5.668166e-01,-7.302657e-02,-5.800451e-01,-3.141633e-01,-7.949237e-02,-3.234127e-01
50%,-3.854784e-01,6.319834e-01,-2.847809e-01,-6.370780e-02,-4.046822e-01,6.935103e-01,-6.150790e-01,-7.640273e-01,-2.805825e-01,3.069605e-01,...,-3.949343e-01,-1.085649e-01,-2.843852e-01,-4.378413e-02,-3.593191e-01,-7.302657e-02,-3.638511e-01,-2.520201e-01,-7.949237e-02,-3.234127e-01
75%,1.005240e+00,6.319834e-01,6.711437e-01,4.969094e-01,4.523594e-01,6.935103e-01,1.625807e+00,1.308854e+00,-2.805825e-01,3.069605e-01,...,-3.245447e-02,-1.085649e-01,1.201950e-01,-4.378413e-02,1.436719e-01,-7.302657e-02,1.500601e-01,-8.468123e-02,-7.949237e-02,-3.234127e-01
max,2.231800e+01,6.319834e-01,4.715440e+00,2.714114e+01,7.723389e+00,6.935103e-01,1.625807e+00,1.308854e+00,3.564014e+00,3.069605e-01,...,2.602861e+01,4.118615e+01,5.611698e+01,1.075882e+02,1.424652e+01,3.666162e+01,1.425628e+01,2.638703e+01,3.539687e+01,3.092025e+00


In [ ]:
# Al no incluir el número de componentes, lo que hace PCA es conservarlos todos
pca = PCA()

In [ ]:
pca.fit(array_scaler)

PCA()

In [ ]:
# Obtén los nombres de las columnas de df4
column_names = df4.columns.tolist()

# Crea una lista de nombres para los índices
index_names = [f'Componente_{i+1}' for i in range(len(column_names))]

# Convierte el array a DataFrame para añadir nombres a los ejes
pca_components_df = pd.DataFrame(
    data=pca.components_,
    columns=column_names,
    index=index_names
)


In [ ]:
print(pca.explained_variance_ratio_)

[6.54291627e-02 5.04237219e-02 4.78500687e-02 4.01359450e-02
 3.71706945e-02 2.96268204e-02 1.80382771e-02 1.69507058e-02
 1.43261298e-02 1.38326281e-02 1.29035973e-02 1.14634604e-02
 1.08302014e-02 1.02660693e-02 9.61697454e-03 9.50749319e-03
 8.98855888e-03 8.57854653e-03 8.30466553e-03 7.88047317e-03
 7.70963450e-03 7.56827547e-03 7.26054068e-03 7.15956008e-03
 6.94090248e-03 6.88582889e-03 6.77820141e-03 6.71173612e-03
 6.62282201e-03 6.61080839e-03 6.55846874e-03 6.42846940e-03
 6.41585044e-03 6.27349622e-03 6.16172391e-03 6.06249576e-03
 5.80309336e-03 5.77816318e-03 5.63335517e-03 5.55659799e-03
 5.47812091e-03 5.44747697e-03 5.42756008e-03 5.34151220e-03
 5.25144827e-03 5.22593569e-03 5.12475997e-03 5.04226973e-03
 4.87817004e-03 4.82069862e-03 4.74091714e-03 4.70392350e-03
 4.56094558e-03 4.48776169e-03 4.41703420e-03 4.33700166e-03
 4.27025203e-03 4.17805311e-03 4.11473636e-03 4.06261303e-03
 3.99156054e-03 3.88110557e-03 3.84839499e-03 3.78805759e-03
 3.67256997e-03 3.627134